In [1]:
!apt-get install tesseract-ocr -y
!pip install pytesseract opencv-python

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
import cv2
import pytesseract
import re
import json

In [3]:
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [5]:
!pip install pdf2image

In [9]:
!apt-get install poppler-utils -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 0s (414 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126308 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...


In [10]:
from pdf2image import convert_from_path
pages = convert_from_path('/content/MTG_10009588.pdf', dpi=300)

In [11]:
image = pages[0]
image.save('page.jpg', 'JPEG')

In [12]:
img = cv2.imread('page.jpg')

In [13]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 15, 8)

In [14]:
custom_config = r'--oem 3 --psm 6'
data = pytesseract.image_to_data(thresh, config=custom_config, output_type=pytesseract.Output.DICT)

In [15]:
n_boxes = len(data['level'])
extracted_data = {}

for i in range(n_boxes):
    text = data['text'][i].strip()
    if text == '' or int(data['conf'][i]) < 50:
        continue  # Skip low confidence results

    (x, y, w, h) = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])

In [23]:
if re.search(r'\$\d{1,3}(,\d{3})*(\.\d{2})?', text):
        extracted_data['Loan Amount'] = {'value': text, 'bounding_box': [x, y, w, h]}
elif re.search(r'\d{1,2}/\d{1,2}/\d{4}', text):
        extracted_data['Loan Date'] = {'value': text, 'bounding_box': [x, y, w, h]}
elif text.lower() in ['mortgage', 'deed', 'deed of trust']:
        extracted_data['Document Title'] = {'value': text, 'bounding_box': [x, y, w, h]}

In [24]:
for i in range(n_boxes):
    if int(data['conf'][i]) > 50:
        (x, y, w, h) = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

In [25]:
cv2.imwrite('output.jpg', img)

True

In [26]:
print(json.dumps(extracted_data, indent=4))

{}
